In [2]:
import numpy as np
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
import cv2
import os
import glob

import functions as fn

In [3]:
dataDir='../data'
trainDir='train2017'
valDir='val2017'
seed=42
transformed=.5
train=.8
test = 1 - train
strategy_list = ['left', 'right']
destDataDir='../created_data'
destTrainDir='train'
destTestDir='test'
destValDir='val'
destNormDir='NORMAL'
destRSDir='ROLLING_SHUTTER'

np.random.seed(seed)

trainAnn=f'{dataDir}/annotations/instances_{trainDir}.json'
valAnn=f'{dataDir}/annotations/instances_{valDir}.json'

cocoTrain=COCO(trainAnn)
cocoVal=COCO(valAnn)

catIdsTrain = cocoTrain.getCatIds(catNms=['person'])
imgIdsTrain = cocoTrain.getImgIds(catIds=catIdsTrain)
imgIdsTrain = cocoTrain.getImgIds(imgIds=imgIdsTrain)
annIdsTrain = cocoTrain.getAnnIds(imgIds=imgIdsTrain, catIds=catIdsTrain, iscrowd=None)

catIdsVal = cocoVal.getCatIds(catNms=['person'])
imgIdsVal = cocoVal.getImgIds(catIds=catIdsVal)
imgIdsVal = cocoVal.getImgIds(imgIds=imgIdsVal)
annIdsVal = cocoVal.getAnnIds(imgIds=imgIdsVal, catIds=catIdsVal, iscrowd=None)

loading annotations into memory...
Done (t=10.40s)
creating index...
index created!
loading annotations into memory...
Done (t=0.41s)
creating index...
index created!


In [7]:
count = 0
limit = 10

# 69 as control
# 42 works
# 7 works


idx = 42

testImg = cocoTrain.loadImgs(imgIdsTrain[idx])[0]
testAnn = cocoTrain.loadAnns(cocoTrain.getAnnIds(imgIds=testImg['id'], catIds=catIdsTrain, iscrowd=None))

fpath = '{}/{}/{}'.format(dataDir, trainDir, testImg['file_name'])

testImgArr = cv2.imread(fpath)

fn.remove_true_blacks(testImgArr)

whichAnn = np.random.randint(0, len(testAnn))

testSpecAnn = testAnn[whichAnn] # np.random.randint(0, len(testAnn))
strategy = strategy_list[np.random.randint(0, len(strategy_list))]

strategy='left'

print(whichAnn, strategy)

maskedTest, maskedTestInv = fn.generate_masked_images(testImgArr, testSpecAnn)

newMaskedTest = fn.apply_rolling_shutter(maskedTest, testSpecAnn, strategy=strategy)

newMaskedTestInv = fn.fill_inv_masked(maskedTestInv, testSpecAnn, strategy=strategy)

final_output = fn.recombine_masked_imgs(newMaskedTest, newMaskedTestInv)

fn.display_image(final_output)

0 left


In [8]:
#cv2.imwrite('modified.png', final_output)
#cv2.imwrite('original.png', testImgArr)

True

In [5]:
type(imgIdsTrain[idx])

int

In [6]:
np.random.seed(seed)

prog = 0
iters = 0
limit = 200 # number of images to process for both RS and normal; results in double the amount (limit of 200 = 400 total images)

ids_ignore = [126798, 44140, 502360, 42779, 506987]

destDir = os.path.join(os.getcwd(), destDataDir.replace('/', '\\'))
for traintestval in [destTrainDir, destTestDir, destValDir]:
    for normrs in [destNormDir, destRSDir]:
        files = glob.glob(f'{destDir}\\{traintestval}\\{normrs}\\*')
        for f in files:
            os.remove(f)

### ROLLING SHUTTER

ids_list = []
for i in np.arange(limit):
    id = np.random.choice(imgIdsTrain)
    if int(id) not in ids_ignore:
        ids_list.append(id)

for id in ids_list: # rolling shutter
    print(f'Loading image id: {id}')

    imgDict = cocoTrain.loadImgs([id])[0]
    annDict = cocoTrain.loadAnns(cocoTrain.getAnnIds(imgIds=imgDict['id'], catIds=catIdsTrain, iscrowd=None))

    fpath = '{}/{}/{}'.format(dataDir, trainDir, imgDict['file_name'])
    img = cv2.imread(fpath)
    fn.remove_true_blacks(img)

    ann = annDict[np.random.randint(0, len(annDict))]
    strategy = strategy_list[np.random.randint(0, len(strategy_list))]

    masked, maskedInv = fn.generate_masked_images(img, ann)

    masked_rs = fn.apply_rolling_shutter(masked, ann, strategy=strategy)

    maskedInv_filled = fn.fill_inv_masked(maskedInv, ann, strategy=strategy)

    final = fn.recombine_masked_imgs(masked_rs, maskedInv_filled)

    if(prog < train):
        fpath = '{}/{}/{}/{}'.format(destDataDir, destTrainDir, destRSDir, f'rs-{iters}.png')
        cv2.imwrite(fpath, final)
    #elif(prog < train + val):
    #    fpath = '{}/{}/{}/{}'.format(destDataDir, destValDir, destRSDir, f'rs-{iters}.png')
    #    cv2.imwrite(fpath, final)
    else:
        fpath = '{}/{}/{}/{}'.format(destDataDir, destTestDir, destRSDir, f'rs-{iters}.png')
        cv2.imwrite(fpath, final)

    prog += 1 / len(ids_list)
    iters += 1
    print(f'Finished loading! {50*prog:.2f}% done')

### NORMAL

prog = 0
iters = 0
ids_list = []
for i in np.arange(limit):
    ids_list.append(np.random.choice(imgIdsTrain))

for id in ids_list: # rolling shutter
    print(f'Loading image id: {id}')

    imgDict = cocoTrain.loadImgs([id])[0]
    annDict = cocoTrain.loadAnns(cocoTrain.getAnnIds(imgIds=imgDict['id'], catIds=catIdsTrain, iscrowd=None))

    fpath = '{}/{}/{}'.format(dataDir, trainDir, imgDict['file_name'])
    img = cv2.imread(fpath)
    fn.remove_true_blacks(img)

    if(prog < train):
        fpath = '{}/{}/{}/{}'.format(destDataDir, destTrainDir, destNormDir, f'rs-{iters}.png')
        cv2.imwrite(fpath, img)
    #elif(prog < train + val):
    #    fpath = '{}/{}/{}/{}'.format(destDataDir, destValDir, destNormDir, f'rs-{iters}.png')
    #    cv2.imwrite(fpath, img)
    else:
        fpath = '{}/{}/{}/{}'.format(destDataDir, destTestDir, destNormDir, f'rs-{iters}.png')
        cv2.imwrite(fpath, img)

    prog += 1 / len(ids_list)
    iters += 1
    print(f'Finished loading! {50 + 50*prog:.2f}% done')


Loading image id: 375637
Finished loading! 0.26% done
Loading image id: 290899
Finished loading! 0.51% done
Loading image id: 263700
Finished loading! 0.77% done
Loading image id: 72024
Finished loading! 1.03% done
Loading image id: 389649
Finished loading! 1.28% done
Loading image id: 239558
Finished loading! 1.54% done
Loading image id: 217936
Finished loading! 1.79% done
Loading image id: 544933
Finished loading! 2.05% done
Loading image id: 371910
Finished loading! 2.31% done
Loading image id: 535820
Finished loading! 2.56% done
Loading image id: 161673
Finished loading! 2.82% done
Loading image id: 520978
Finished loading! 3.08% done
Loading image id: 331844
Finished loading! 3.33% done
Loading image id: 433007
Finished loading! 3.59% done
Loading image id: 354699
Finished loading! 3.85% done
Loading image id: 254577
Finished loading! 4.10% done
Loading image id: 347570
Finished loading! 4.36% done
Loading image id: 384422
Finished loading! 4.62% done
Loading image id: 553436
Fini

In [19]:
print(375637 in imgIdsTrain)

True


In [20]:
cocoTrain.loadImgs([375637])

[{'license': 1,
  'file_name': '000000375637.jpg',
  'coco_url': 'http://images.cocodataset.org/train2017/000000375637.jpg',
  'height': 456,
  'width': 640,
  'date_captured': '2013-11-19 02:10:13',
  'flickr_url': 'http://farm8.staticflickr.com/7311/8927750208_b552aa760e_z.jpg',
  'id': 375637}]